In [1]:
import pandas as pd
import numpy as np

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [2]:
import random as rnd

file_name_0 = "data/data_int.txt"
n = 10
list_int = []

for i in range(n):
    list_int.append(rnd.randint(1,11))
    n += 1

# mode can be specified for writing, reading or both
with open(file_name_0, mode = 'w') as f:
    for i in list_int:
        # write each item on a new line
        f.write("%s\n" % i)
    f.close()
    
!cat data/data_int.txt

3
8
5
2
11
10
5
1
1
1


In [3]:
import numpy.random as npr

matrix = npr.rand(5, 5)

file_name_1 = "data/data_float.txt"

with open(file_name_1, mode = 'w') as f:
    for x in np.nditer(matrix):
        # write each item on a new line
        f.write("%s\n" % x)
    f.close()
    
!cat data/data_float.txt

0.3087381399837399
0.9245581840080184
0.47356379128524406
0.1771659777014728
0.8674001704009788
0.26892843437379743
0.3081773136450001
0.363168546100034
0.8830706567999346
0.46722319529347944
0.1426752201656416
0.14341931553724863
0.619491631907853
0.45057443119967266
0.3173493182217717
0.20087838540642533
0.3588485779989151
0.9226581951877726
0.9124566489604882
0.4321542829454694
0.11758486380576472
0.6832381486735393
0.9870996631121843
0.8553611202972347
0.5416850955392749


In [4]:
import csv

file_name_1 = "data/data_float.txt"
file_name_2 = "data/data_float.csv"

with open(file_name_1, 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split("\n") for line in stripped if line)
    with open(file_name_2, 'w') as out_file:
        writer = csv.writer(out_file, delimiter = ',', lineterminator=',\n')
        writer.writerows(lines)
    in_file.close()
    out_file.close()
        
!cat data/data_float.csv

0.3087381399837399,
0.9245581840080184,
0.47356379128524406,
0.1771659777014728,
0.8674001704009788,
0.26892843437379743,
0.3081773136450001,
0.363168546100034,
0.8830706567999346,
0.46722319529347944,
0.1426752201656416,
0.14341931553724863,
0.619491631907853,
0.45057443119967266,
0.3173493182217717,
0.20087838540642533,
0.3588485779989151,
0.9226581951877726,
0.9124566489604882,
0.4321542829454694,
0.11758486380576472,
0.6832381486735393,
0.9870996631121843,
0.8553611202972347,
0.5416850955392749,


2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [5]:
#!wget https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json -P data/

#!cat data/user_data.json

import json # import the JSON module

file_in = "./data/user_data.json"
file_out = "./data/user_data.csv"

data = json.load(open(file_in))

list_filtered = list(filter(lambda x: x["CreditCardType"]=="American Express",data))

keys = list_filtered[0].keys()

with open(file_out, 'w', newline='') as csv_file:
    dict_writer = csv.DictWriter(csv_file, fieldnames = keys)
    dict_writer.writeheader()
    dict_writer.writerows(list_filtered)
    
!cat data/user_data.csv

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [6]:
#!wget https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv -P data/

file_name = "./data/mushrooms_categorized.csv"
data = pd.read_csv(file_name)
data

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


In [7]:
data.groupby('class').mean()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
class,,,,,,,,,,,,,,,,,,,,,
0,3.266160,1.615970,4.581749,0.653992,4.334601,0.954373,0.285171,0.068441,6.622624,0.615970,...,1.798479,6.098859,6.064639,0.0,1.931559,1.125475,3.007605,3.201521,3.283270,1.148289
1,3.436159,2.055158,4.421859,0.159346,3.940756,0.995403,0.028601,0.567926,2.863636,0.514811,...,1.394280,5.512768,5.504597,0.0,2.002043,1.009193,1.522983,4.021450,4.031665,1.895812


In [9]:
data.to_json("./data/mushrooms_categorized.json")

!cat data/mushrooms_categorized.csv

class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1
0,5,3,9,1,0,1,0,0,5,0,2,2,2,7,7,0,2,1,4,2,2,1
0,0,2,8,1,0,1,0,0,2,0,2,2,2,7,7,0,2,1,4,2,2,3
0,0,3,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,3,3
1,5,3,8,1,6,1,0,1,7,0,3,2,2,7,7,0,2,1,4,2,4,1
0,0,2,9,1,0,1,0,0,2,0,2,2,2,7,7,0,2,1,4,2,3,3
0,5,3,9,1,3,1,0,0,2,0,2,2,2,7,7,0,2,1,4,3,2,1
0,5,3,9,1,0,1,0,0,5,0,2,2,2,7,7,0,2,1,4,2,3,3
0,0,2,9,1,0,1,0,0,10,0,2,2,2,7,7,0,2,1,4,3,3,1
1,5,3,8,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,3,4,5
0,5,0,4,0,5,1,1,0,5,1,3,2,0,7,7,0,2,1,0,2,0,1
0,4,0,3,0

1,2,3,8,1,5,1,0,0,2,0,1,2,2,7,7,0,2,2,4,5,4,1
0,3,2,2,1,5,1,0,0,1,0,0,2,2,7,2,0,2,2,0,7,1,6
1,2,3,9,0,2,1,0,0,2,0,1,1,1,4,4,0,2,1,2,1,4,4
0,3,2,4,1,5,1,0,0,1,0,0,2,2,7,2,0,2,2,0,7,1,6
1,2,3,9,0,5,1,0,1,10,0,0,1,3,7,4,0,2,1,0,7,4,0
1,2,3,3,0,2,1,0,0,2,0,1,1,1,6,4,0,2,1,2,1,5,4
1,2,3,9,0,2,1,0,0,2,0,1,1,1,6,0,0,2,1,2,1,4,1
1,5,3,2,0,8,1,0,1,0,1,0,1,2,6,7,0,2,1,0,7,4,4
0,2,2,0,1,5,1,0,0,10,0,0,2,2,2,2,0,2,2,0,7,1,6
1,2,2,8,1,2,1,0,0,10,1,1,0,2,7,7,0,2,1,4,1,4,5
1,5,2,8,1,2,1,0,0,10,1,1,0,0,7,7,0,2,1,4,1,3,1
1,2,3,3,0,2,1,0,0,3,0,1,1,1,6,6,0,2,1,2,1,5,0
0,2,3,5,1,5,1,0,0,10,0,0,2,2,2,7,0,2,2,0,7,1,6
1,2,3,9,0,2,1,0,0,7,0,1,1,1,4,4,0,2,1,2,1,5,1
1,2,3,9,0,2,1,0,0,7,0,1,1,1,4,4,0,2,1,2,1,5,4
0,3,2,2,1,5,1,0,0,10,0,0,2,2,2,2,0,2,2,0,7,1,6
0,5,3,4,1,5,1,0,0,1,0,0,2,2,2,2,0,2,2,0,7,1,6
1,5,3,9,0,2,1,0,0,3,0,1,1,1,6,4,0,2,1,2,1,4,1
1,2,2,0,1,2,1,0,0,7,1,1,2,0,7,7,0,2,1,4,1,4,1
1,5,3,3,0,2,1,0,0,2,0,1,1,1,6,6,0,2,1,2,1,4,0
1,2,2,3,1,2,1,0,0,10,1,1,0,0,7,7,0,2,1,4,1,3,5
1,5,2,8,1,2,1,0,0,7,1,1,0,2

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [10]:
#!wget https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat -P data/

infile = 'data/credit_card.dat'

with open(infile, 'r') as f:
    card_number = ''
    for row in f:
        block = ''
        for i in range(0, len(row), 6):
            block += chr(int(row[i:i+6],2))
        card_number += block

print(card_number)

7648 5673 3775 2271
3257 8247 3354 2266
2722 0001 4011 6652
0661 3063 3742 3150
0432 1608 1462 4742
5827 2027 8785 7303
5774 8528 2087 1117
8140 1210 6352 2845
5764 1133 7301 7100
6456 1737 4126 6726
1228 8631 7382 0000
7051 0160 5374 3166
0618 3587 1630 6376
1545 5454 7444 5636
6735 3116 3202 6834
7287 5011 1547 8413
7033 2607 3328 4200
2568 5244 1874 5024
1684 2253 7570 7118
0672 2576 0575 6631
6332 8353 8787 1340
1813 3361 1175 4211
2477 6450 8840 2368
5512 3505 2563 1326
3083 7882 0621 0025
4521 5148 8045 0334
7563 3654 8713 5787
8324 2664 0476 5561
0565 2504 7168 3510
5107 5507 1767 0738
2462 1821 2448 1443
2788 0638 6861 6554
5851 5873 5474 0547
0670 1004 4013 2655
5874 5506 3048 0806
2805 5401 8462 1260
5083 8406 6310 1862
1076 1445 3013 2266
8440 4804 4844 5277
4758 6141 0686 1387
7586 0675 0315 2568
2544 1258 7432 5165
3474 5023 4434 5626
1410 0270 0434 5086
7315 4446 1104 4215
0224 7742 8300 0266
0170 2700 3145 0640
2006 2437 8054 1600
8142 4055 1776 0026
3026 7380 1241 1084


5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?

In [11]:
import pandas as pd

file_name = "./data/data_000637.txt"
df = pd.read_csv(file_name, nrows=10)
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
5,1,0,63,3869200167,2762,4
6,1,0,61,3869200167,2772,14
7,1,0,139,3869200167,2776,0
8,1,0,62,3869200167,2774,21
9,1,0,60,3869200167,2788,7


In [12]:
import struct, time

for index, row in df.iterrows():
    
    head_shift = row['HEAD'] << 62
    fpga_shift = row['FPGA'] << 58 
    tdc_channel_shift = row['TDC_CHANNEL'] << 49
    orbit_cnt_shift = row['ORBIT_CNT'] << 17
    bx_counter_shift = row['BX_COUNTER'] << 5
    tdc_meas_shift = row['TDC_MEAS'] << 0
    word = head_shift|fpga_shift|tdc_channel_shift|orbit_cnt_shift|bx_counter_shift|tdc_meas_shift
    
    print(bin(word))

0b100000011110111110011010011111010011110010011110010100011011010
0b100000011111001110011010011111010011110010011110010100011011011
0b100000001111111110011010011111010011110010011110011111100111100
0b100000010000001110011010011111010011110010011110011111111010011
0b100000010000001110011010011111010011110010011110101100100011001
0b100000001111111110011010011111010011110010011110101100101000100
0b100000001111011110011010011111010011110010011110101101010001110
0b100000100010111110011010011111010011110010011110101101100000000
0b100000001111101110011010011111010011110010011110101101011010101
0b100000001111001110011010011111010011110010011110101110010000111


In [13]:
with open('data/binary_file_data_000637.dat', 'wb') as binary_file:
    for index, row in df.iterrows():
        head_shift = row['HEAD'] << 62
        fpga_shift = row['FPGA'] << 58 
        tdc_channel_shift = row['TDC_CHANNEL'] << 49
        orbit_cnt_shift = row['ORBIT_CNT'] << 17
        bx_counter_shift = row['BX_COUNTER'] << 5
        tdc_meas_shift = row['TDC_MEAS'] << 0
        word = head_shift|fpga_shift|tdc_channel_shift|orbit_cnt_shift|bx_counter_shift|tdc_meas_shift #int64
        binary_file.write(struct.pack('<q', word))

binary_file.close()

# !cat data/binary_file_data_000637.dat

In [14]:
import struct, time

data = {}
columns = ['HEAD', 'FPGA', 'TDC_CHANNEL', 'ORBIT_CNT', 'BX_COUNTER', 'TDC_MEAS']
df_new = pd.DataFrame({}, columns=columns)

with open('data/binary_file_data_000637.dat', 'rb') as file:
    file_content = file.read()
    word_counter = 0
    word_size = 8 # size of the word in bytes
    for i in range(0, len(file_content), word_size):
        word_counter += 1
        if word_counter > 10: break
        word = struct.unpack('<q', file_content[i : i + word_size])[0] # get an 8-byte word
        head     = (word >> 62) & 0x3
        fpga     = (word >> 58) & 0xF
        tdc_chan = (word >> 49) & 0x1FF
        orb_cnt  = (word >> 17) & 0xFFFFFFFF
        bx       = (word >> 5 ) & 0xFFF
        tdc_meas = (word >> 0 ) & 0x1F
        #if i == 0: print ('{0}\t{1}\t{2}\t{3}\t{4}\t{5}'.format('HEAD', 'FPGA', 'CHANNEL', 'ORBIT_CNT', 'BX_CNT', 'TDC_MEAS'))
        #print('{0}\t{1}\t{2}\t{3}\t{4}\t{5}'.format(head, fpga, tdc_chan, orb_cnt, bx, tdc_meas))
        entry = {'HEAD' : head, 'FPGA' : fpga, 'TDC_CHANNEL' : tdc_chan, 'ORBIT_CNT' : orb_cnt, 'BX_COUNTER' : bx, 'TDC_MEAS' : tdc_meas}
        df_new = df_new.append(entry, ignore_index=True)

df_new

/tmp/ipykernel_1508/230043877.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(entry, ignore_index=True)
/tmp/ipykernel_1508/230043877.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(entry, ignore_index=True)
/tmp/ipykernel_1508/230043877.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(entry, ignore_index=True)
/tmp/ipykernel_1508/230043877.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(entry, ignore_index=True)
/tmp/ipykernel_1508/230043877.py:24: FutureWarning: The frame.append method is deprecated and will be removed from p

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
5,1,0,63,3869200167,2762,4
6,1,0,61,3869200167,2772,14
7,1,0,139,3869200167,2776,0
8,1,0,62,3869200167,2774,21
9,1,0,60,3869200167,2788,7


In [15]:
import os

binary_file = 'data/binary_file_data_000637.dat'

file_stats = os.stat(binary_file)

print(file_stats)
print(f'File Size in Bytes is {file_stats.st_size}')
print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

os.stat_result(st_mode=33188, st_ino=74185, st_dev=2064, st_nlink=1, st_uid=1000, st_gid=1000, st_size=80, st_atime=1670016864, st_mtime=1670016856, st_ctime=1670016856)
File Size in Bytes is 80
File Size in MegaBytes is 7.62939453125e-05
